# Exploración de Dataset: Balance Energético

## Importación de librerías y seteo de variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import openpyxl as pyxl
from __future__ import print_function
from __future__ import unicode_literals
# matplotlib defaults
%matplotlib inline
matplotlib.style.use("ggplot")
import os

In [ ]:
FICHEROS = {"microdatos": "datos/Datos Abiertos Series V2a Original.xlsx",
            "sintesis": "datos/Resumen de los Datos.xlsx",
            "poblacion": "datos/serie_poblacion.csv"}

## Carga de datos

### Serie de población

In [ ]:
df_pobl = pd.read_csv(FICHEROS["poblacion"],index_col="anio")
poblacion = pd.Series(data=df_pobl["poblacion"], index=df_pobl.index)

In [ ]:
poblacion.plot(figsize=(14, 7))

### Parseo de microdatos de energía

A partir de la planilla de microdatos de Balance Energético, se genera un Panel de datos con los siguientes ejes:
* Items axis (**0**, DataFrames por año): 1960 to 2015
* Major_axis axis (**1**, Índice por Energía): ACEITES VEGETALES to SOLAR
* Minor_axis axis (**2**, Columnas por Uso): PROD to INDUS

In [ ]:
wb = pyxl.load_workbook(FICHEROS["microdatos"], data_only=True)

In [ ]:
def sheet_to_df(ws):
    raw_data = ws.values
    # La celda A1 tiene el nombre de la energía de la planilla
    name = next(raw_data)[0]
    cols = next(raw_data)
    rows = list(raw_data)[:56] # Luego de 56 filas de años, algunas sheets tienen aclaraciones
    df = pd.DataFrame.from_records(rows, columns=cols)
    # Asigno índice y corrijo el nombre
    df = df.set_index("KTEP")
    df.index = df.index.rename("anio")
    # Elimino las columnas vacías
    df = df.drop(None, axis=1)
    # Relleno None y NaN con ceros
    df = df.fillna(0)
    return df, name

In [ ]:
ENERGIAS = {}
for ws in wb.worksheets:
    df, energia = sheet_to_df(ws)
    ENERGIAS[energia] = df

In [ ]:
pan_crudo = pd.Panel(ENERGIAS)
pan_crudo = pan_crudo.transpose(1, 0, 2)
pan_crudo

In [ ]:
pan.loc[:,:,"PROD+IMPO"] = pan[:,:,"PROD"] + pan[:,:,"IMPO"]

In [ ]:
pan[:,:,"PROD+IMPO"]